In [1]:
import pymc as pm
import numpy as np
import arviz as az
from pymc.math import matrix_inverse, extract_diag, sqrt
import aesara.tensor as at

%load_ext lab_black
%load_ext watermark

# Dental Development


Adapted from [unit 10: growth.odc](https://raw.githubusercontent.com/areding/6420-pymc/main/original_examples/Codes4Unit10/growth.odc).

Data for the y array can be found [here](https://raw.githubusercontent.com/areding/6420-pymc/main/data/growthy.txt).

Associated lecture video: Unit 10 Lesson 2

## Problem statement


Data set on dental development was first provided by Potthof and Roy in their 1964 paper. It consists of longitudinal observations on 11 girls (gender=1) and 16 boys (gender=2). 

There are 4 observations on each subject centered at times -3,-1, 1, 3, where the units were years.

The measurement on each subject is the distance (in mm) from the center of the pituitary to the pteryomaxillary fisure.

Potthoff and Roy (1964). "A Generalized Multivariate Analysis of Variance Model Useful Especially for Growth Curve Problems," Biometrika, 51, 313-326.

MVN with Gender Specific Means but Common Precision Matrix

notes:
- definitely talk about Wishart. Pymc docs say it's unusable??? why tf do they have it?
- https://github.com/pymc-devs/pymc/issues/538 super interesting discussion here
- this is all new to me, need to think about it more.

In [3]:
t = np.array((-3, -1, 1, 3))
gender = np.array(
    (1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2)
)
y = np.loadtxt("../data/growthy.txt")

In [4]:
C = np.eye(4)
C

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

In [5]:
coords = {"t": t, "gender": np.array([1, 2]), "id": np.array(range(26))}

In [6]:
coords

{'t': array([-3, -1,  1,  3]),
 'gender': array([1, 2]),
 'id': array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25])}

In [7]:
pm.Wishart.dist(nu=4, V=C).eval()

/Users/aaron/mambaforge/envs/pymc_env/lib/python3.10/site-packages/pymc/distributions/multivariate.py:936: UserWarning: The Wishart distribution can currently not be used for MCMC sampling. The probability of sampling a symmetric matrix is basically zero. Instead, please use LKJCholeskyCov or LKJCorr. For more information on the issues surrounding the Wishart see here: https://github.com/pymc-devs/pymc/issues/538.
  warnings.warn(


array([[ 0.72360432,  1.12109233, -0.94095466,  0.01946187],
       [ 1.12109233,  3.36487782, -0.51769377,  1.38565438],
       [-0.94095466, -0.51769377,  2.56146861, -0.50116026],
       [ 0.01946187,  1.38565438, -0.50116026,  3.82395345]])

In [8]:
pm.LKJCorr.dist(n=4, eta=0.4).eval()

array([-0.20555731, -0.02524332, -0.53060013,  0.94640356,  0.52935778,
        0.62563837])

In [9]:
pm.WishartBartlett(nu=4, S=C).eval()

TypeError: WishartBartlett() missing 1 required positional argument: 'name'

In [ ]:
with pm.Model(coords=coords) as m:
    beta1 = pm.Normal("beta1", 20, tau=.001, dims="gender")
    beta2 = pm.Normal("beta2", 1, tau=.001, dims="gender")
    g = pm.Data("gender_idx", gender, dims="id", mutable=False)

    # T = pm.Wishart("T", nu=4, V=C) # might need to switch to LKJ
    T, corr, stds = pm.LKJCholeskyCov("T", n=4, eta=1, sd_dist=pm.Normal.dist(0, 1), compute_corr=True)

    mu = pm.Deterministic("mu", beta1[g] + beta2[g] * t, dims=("gender", "t")) # mu should be shape (2, 4)? but mvnormal mu should be a vector...
    pm.MvNormal("likelihood", mu, chol=T, observed=y) # (26, 4)

    pm.Deterministic("corr", corr)
    #V = matrix_inverse(T)
    #diag_sqrt = sqrt(extract_diag(V))
    #V = V/diag_sqrt[:, None]
    #corr = pm.Deterministic("corr", V/diag_sqrt[None, :])

    trace = pm.sample(2000)